In [1]:
!pip install spacy textblob
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 93.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
import spacy
from textblob import TextBlob
import pandas as pd

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")


In [7]:
reviews = [
    "I love the Apple iPhone 15, it's amazing!",
    "The Samsung Galaxy S23 has a great camera but poor battery life.",
    "This Sony headphones model WH-1000XM5 is fantastic for noise cancellation.",
    "I regret buying the Dell Inspiron laptop; it keeps overheating.",
    "The Kindle Paperwhite is perfect for reading, very happy with it!"
]

# Known brands and products (for rule-based extraction)
brands = ["Apple", "Samsung", "Sony", "Dell", "Kindle"]
products = ["iPhone 15", "Galaxy S23", "WH-1000XM5", "Inspiron", "Paperwhite"]


In [8]:
data = []

for review in reviews:
    doc = nlp(review)
    blob = TextBlob(review)

    # Sentiment analysis
    polarity = blob.sentiment.polarity
    sentiment = "Positive" if polarity > 0 else "Negative" if polarity < 0 else "Neutral"

    # Extract entities using spaCy
    spacy_brands = [ent.text for ent in doc.ents if ent.label_ == "ORG"]

    # Rule-based extraction for brands and products
    found_brands = list(set(spacy_brands + [b for b in brands if b in review]))
    found_products = [p for p in products if p in review]

    data.append({
        "Review": review,
        "Brands": found_brands,
        "Products": found_products,
        "Sentiment": sentiment
    })

# Create a clean table
df = pd.DataFrame(data)
df


,Review,Brands,Products,Sentiment
0,"I love the Apple iPhone 15, it's amazing!",[Apple],[iPhone 15],Positive
1,The Samsung Galaxy S23 has a great camera but ...,"[Samsung, S23]",[Galaxy S23],Positive
2,This Sony headphones model WH-1000XM5 is fanta...,[Sony],[WH-1000XM5],Positive
3,I regret buying the Dell Inspiron laptop; it k...,"[Dell, Dell Inspiron]",[Inspiron],Neutral
4,"The Kindle Paperwhite is perfect for reading, ...","[Kindle, The Kindle Paperwhite]",[Paperwhite],Positive


In [9]:
negative_words = ["regret", "poor", "overheating", "disappointing", "bad"]

data = []

for review in reviews:
    doc = nlp(review)
    blob = TextBlob(review)

    # Sentiment: stricter negative detection
    if any(word in review.lower() for word in negative_words):
        sentiment = "Negative"
    else:
        polarity = blob.sentiment.polarity
        sentiment = "Positive" if polarity > 0 else "Negative" if polarity < 0 else "Neutral"

    # Extract entities
    spacy_brands = [ent.text for ent in doc.ents if ent.label_ == "ORG"]
    found_brands = list(set(spacy_brands + [b for b in brands if b in review]))

    # Remove duplicates where brand name appears in product
    found_brands = [b for b in found_brands if b in brands]

    found_products = [p for p in products if p in review]

    data.append({
        "Review": review,
        "Brands": found_brands,
        "Products": found_products,
        "Sentiment": sentiment
    })

df = pd.DataFrame(data)
df


,Review,Brands,Products,Sentiment
0,"I love the Apple iPhone 15, it's amazing!",[Apple],[iPhone 15],Positive
1,The Samsung Galaxy S23 has a great camera but ...,[Samsung],[Galaxy S23],Negative
2,This Sony headphones model WH-1000XM5 is fanta...,[Sony],[WH-1000XM5],Positive
3,I regret buying the Dell Inspiron laptop; it k...,[Dell],[Inspiron],Negative
4,"The Kindle Paperwhite is perfect for reading, ...",[Kindle],[Paperwhite],Positive
